# Homework 2: Two-layer neural network
### Due Tuesday, January 29, 2019, 5pm
#### Jesse Zhu
#### ML-W2019

Using the MNIST hand-written digits dataset, we aim to categorize the digits 0-9 with a two-layer neural network,
composed of one hidden layer and one output layer. Both layers will use the sigmoid activation function, and the
network is fully connected.

Sigmoid function: 1/ (1+ e^-z)

In [1]:
#import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#IMPORT DATA#################
testfile = "mnist_test.csv"
trainfile = "mnist_train.csv"

#Data is 785 columns by N rows. First column = Label (0-9), others = 0:255
test_data = np.genfromtxt(testfile, skip_header=True, delimiter=',')
train_data = np.genfromtxt(trainfile, skip_header=True, delimiter=',')

#PREPROCESSING###############

#SCALE to 0:1
test_data[:,1:] /= 255
train_data[:,1:] /= 255

In [28]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

#Accuraccy function
def acc(w1, w2, data, debug = 0):
    """
    input: 
        w1: input to hidden weights [785 by hu]
        w2: hidden to output weights [(hu+1) by 10]
        data: matrix of N data points by 785 values, where the first value is the target (0-9), and the rest are
            scaled gray-scale values between 0 and 1
    output: Percentage of correct classifications, where a classification is taken using the highest perceptron
        output value
    """
    length = len(data[:,0])
    xi = np.append(np.ones([length,1]), data[:, 1:],1) #Append 1s for bias & strip label from data[]
    h_act = np.array(list(map(sigmoid, xi @ w1[:, :]))) #N by (hidden_units)
    hi = np.append(np.ones([length,1]), h_act[:,:],1) #N by (hidden_units+1)
#sigmoid necessary for out?
    o_act = np.array(list(map(sigmoid, hi @ w2[:, :]))) #N by 10

    amax = o_act.argmax(axis=1) #Predicted value
    if debug:
        print(o_act.shape)
        print(amax[0:25])
    ret = np.sum(amax == data[:,0])
    return ret/length

#Confusion matrix
def cm(w1,w2, data):
    """
    input: 
        w1: input to hidden weights [785 by hu]
        w2: hidden to output weights [(hu+1) by 10]
        data: matrix of N data points by 785 values, where the first value is the target (0-9), and the rest are
            scaled gray-scale values between 0 and 1
    output: confusion matrix 11x11 including labels (actual vs predicted)
    """
    length = len(data[:,0])
    xi = np.append(np.ones([length,1]), data[:, 1:],1) #Append 1s for bias & strip label from data[]
    h_act = np.array(list(map(sigmoid, xi @ w1[:, :]))) #N by (hidden_units)
    hi = np.append(np.ones([length,1]), h_act[:,:],1) #N by (hidden_units+1)
#sigmoid necessary for out?
    o_act = np.array(list(map(sigmoid, hi @ w2[:, :]))) #N by 10
    
    m = np.zeros([11,11])
    m[0,1:] = np.arange(10)
    m[1:,0] = np.arange(10)
    
    for i in range(0, length):
        actual = int(data[i, 0])
        predic = int((o_act[i,:]).argmax(axis=0))
        m[actual+1, predic+1] += 1
        
    return m.astype(int)

In [31]:
epochs = 1
learning_rate = 0.1
momentum = 0.9
hidden_units = [10, 20, 100]


for hu in hidden_units:
    print("***# Hidden = ",hu,"***************")
    w_hi = (np.random.rand(785,hu) * 0.10) - 0.05 #weights, input to hidden
    w_oh = (np.random.rand(hu+1,10) * 0.10) - 0.05 #weights, hidden to output
    acc_test = []
    acc_train = []
    print(acc(w_hi, w_oh, test_data))
    print(cm(w_hi, w_oh, test_data))

***# Hidden =  10 ***************
0.11351135113511351
[[   0    0    1    2    3    4    5    6    7    8    9]
 [   0    0  980    0    0    0    0    0    0    0    0]
 [   1    0 1135    0    0    0    0    0    0    0    0]
 [   2    0 1032    0    0    0    0    0    0    0    0]
 [   3    0 1010    0    0    0    0    0    0    0    0]
 [   4    0  982    0    0    0    0    0    0    0    0]
 [   5    0  892    0    0    0    0    0    0    0    0]
 [   6    0  958    0    0    0    0    0    0    0    0]
 [   7    0 1027    0    0    0    0    0    0    0    0]
 [   8    0  974    0    0    0    0    0    0    0    0]
 [   9    0 1009    0    0    0    0    0    0    0    0]]
***# Hidden =  20 ***************
0.09740974097409741
[[   0    0    1    2    3    4    5    6    7    8    9]
 [   0    0    0    0    0    0    0    0    0  980    0]
 [   1    0    0    0    0    0    0    0    0 1135    0]
 [   2    0    0    0    0    0    0    0    0 1032    0]
 [   3    0    0    0

In [24]:
test1 = np.zeros(5)
test2 = np.append(1, test1)
print(list(map(sigmoid,[0.1, 0.2, 0.3, 0.4, 0.8, 0.9])))

[0.52497918747894, 0.549833997312478, 0.574442516811659, 0.598687660112452, 0.6899744811276125, 0.7109495026250039]
